# Comparative Stock Market Analysis in R using Quandl & tidyverse- Part I

## Introduction

https://www.analyticsvidhya.com/blog/2017/09/comparative-stock-analysis/

What differentiates the best data scientists from others? It is their focus on application of data science. The best data scientists I know of, see data science and its application every where they look. They look at this world as an outcome of flow of data and information.

On the other hand, most beginners often ask the question – how do we apply our learning on real life problems?

In this post (and another one following this), I have picked up a real life dataset (Stock Markets in India) and showed how I would use this data to come out with useful insights.

I hope that you will find this useful. The idea is show the vast opportunities present in data science in a simple yet powerful manner. If you can think of more examples like this – let me know in comments below!

For the best results, I would strongly recommend to build the application yourself as you follow the tutorial.

## Objective of this Tutorial

In this article, we will analyze stock market in banking segment based on the bank stocks which are listed in NSE India. Our objective is to find the trends (Seasonal or cyclic) in banking stocks.

In our comparative analysis we will use several packages and the primary focus will be on tidy verse package. The emphasis will be given on grouping with the help of **tibble** dataframe from tidy verse package. This will help to perform similar operation on multiple groups at a time, hence reducing the code length and computational time

This article also focuses on API Key, database code search using quandl, and finally how to directly download the data from R Console

So lets get started!

Note: The code that has been mentioned below is to be run on the R command line for best results.

Table of Contents
* Setting up the system
* Getting Started with Comparative Analysis
* * Creating the dataset
* * Visualizing the monthly prices
* * Discovering the Relation between Total Traded Quantity vs Close Price
* * Finding the Density Distribution of Deviation of High Price from Open Price
* * Observing the Autocorrelation lags

### Setting Up The System

There are a few things you should take care of before you go on further. Below mentioned are the packages you need to install in the system

- Quandl for Data Download
- timetk to coerce the dataframe into xts
- tidyverse to Use tibble for grouping and perform single operation on multiple groups
- tidyquant for Time Series and Financial functions to perform the analysis
- gglot for Plotting and Visualization
- gganimate to plot the monthly prices. To get more information on gganimate, please read my previous post on Analytics Vidhya
- forcats for modification of factor levels
- stringr for string use

In [17]:
library(plyr)

In [7]:
library(zoo)

In [8]:
library(Quandl)

In [10]:
library(tidyverse)

In [11]:
library(timetk)

Warning message:
"package 'timetk' was built under R version 3.4.2"

In [12]:
library(forcats)

Warning message:
"package 'forcats' was built under R version 3.4.2"

In [13]:
library(stringr)

In [14]:
library(gganimate)

In [18]:
library(dplyr)

In [19]:
library(stringr)

In [20]:
library(gridExtra)


Attaching package: 'gridExtra'

The following object is masked from 'package:dplyr':

    combine



In [305]:
library(plotly)


Attaching package: 'plotly'

The following objects are masked from 'package:plyr':

    arrange, mutate, rename, summarise

The following object is masked from 'package:ggplot2':

    last_plot

The following object is masked from 'package:stats':

    filter

The following object is masked from 'package:graphics':

    layout



### Creating the Dataset
We will be using Quandl is online repository for the core financial, macroeconomic statistics and forex. Quandl has a vast collection of free and open data collected from a variety of organizations: central banks, governments, multinational organizations and more. You can use it without payment and with few restrictions.

Both Free and Premium data are available. Authenticated Free users have a limit of 300 calls per 10 seconds, 2,000 calls per 10 minutes and a limit of 50,000 calls per day. Premium data subscribers have a limit of 5,000 calls per 10 minutes and a limit of 720,000 calls per day.

We will use this online repository to get our data using “Quandl” package directly from the R Console. Quandl package directly interacts with the Quandl API to offer data in a number of formats usable in R, downloading a zip with all data from a Quandl database, and the ability to search.

For More information on Quandl Package, please visit this page. https://cran.r-project.org/web/packages/Quandl/Quandl.pdf

To get started with Quandl, create an account and get the quandl API key. Please click here to create an account. Then click on the Login button provided on the top right corner of the screen. Once the registration is complete, please click here to get the API Key.

Login : https://www.quandl.com/
API Key : https://www.quandl.com/account/api

unpw: my gmail uspw

In [21]:
## Setup the Quandl Free Account and API Key, Please copy and paste the API key in order to #authenticate
Quandl.api_key("rMrxniUJH1-pmm4edrqA")

#### We will use MCX database with Lead commodity for analysis

[MCX-Multi-Commodity-Exchange-India](www.quandl.com/data/MCX-Multi-Commodity-Exchange-India)


You see there are Multiple data sets we have for Lead. Every contact is a seperate Data set. 
for  example
- Lead Futures, October 2014, **PBV2014**, MCX
- Lead Futures, February 2016, **PBG2016**, MCX

We will have to get all those individual data sets seperatly and merge them as one set 

** Serach for Lead for last 36 contacts or 3 years of data**

In [244]:
MCX_Search_Results = Quandl.search("Lead Futures",per_page = 50,silent = TRUE)

In [245]:
MCXFuturesMeta_DF = data.frame(Name = MCX_Search_Results$name, Exchange = MCX_Search_Results$database_code,DataSetCode = MCX_Search_Results$dataset_code,ContractDate=as.Date(str_c('01 ',str_replace(word(MCX_Search_Results$name,3,4),",","")),format = '%d %B %Y'),ContractExpiry=str_replace(word(MCX_Search_Results$name,3,4),",",""))

In [246]:
MCXFuturesMeta_DF = MCXFuturesMeta_DF[order(MCXFuturesMeta_DF$ContractDate),]

In [247]:
MCXFuturesMeta_DF

,Name,Exchange,DataSetCode,ContractDate,ContractExpiry
31,"Lead Futures, December 2013, PBZ2013, MCX",MCX,PBZ2013,2013-12-01,December 2013
41,"Lead Futures, January 2014, PBF2014, MCX",MCX,PBF2014,2014-01-01,January 2014
13,"Lead Futures, February 2014, PBG2014, MCX",MCX,PBG2014,2014-02-01,February 2014
14,"Lead Futures, March 2014, PBH2014, MCX",MCX,PBH2014,2014-03-01,March 2014
42,"Lead Futures, April 2014, PBJ2014, MCX",MCX,PBJ2014,2014-04-01,April 2014
12,"Lead Futures, June 2014, PBM2014, MCX",MCX,PBM2014,2014-06-01,June 2014
43,"Lead Futures, July 2014, PBN2014, MCX",MCX,PBN2014,2014-07-01,July 2014
3,"Lead Futures, August 2014, PBQ2014, MCX",MCX,PBQ2014,2014-08-01,August 2014
15,"Lead Futures, September 2014, PBU2014, MCX",MCX,PBU2014,2014-09-01,September 2014
1,"Lead Futures, October 2014, PBV2014, MCX",MCX,PBV2014,2014-10-01,October 2014


In [248]:
#Initial Set-up of the Data
temp0_code = str_c("MCX/",MCXFuturesMeta_DF$DataSetCode[1])
temp0 = Quandl(temp0_code,type = "raw",collapse = "daily")
temp0["ContractDate"] = MCXFuturesMeta_DF$ContractDate[1]
temp0["ContractExpiry"] = MCXFuturesMeta_DF$ContractExpiry[1]
temp0 = temp0[-c(1:nrow(temp0)),]

In [249]:
temp0

Date,Open,High,Low,Close,Volume,Prev. Day Open Interest,ContractDate,ContractExpiry


In [250]:
i=0

for (k in MCXFuturesMeta_DF$DataSetCode){
    MCX_code = str_c("MCX/",k)
    i=i+1
    temp1 = Quandl(MCX_code,type = "raw",collapse = "daily")
    temp1["ContractDate"] = MCXFuturesMeta_DF$ContractDate[i]
    temp1["ContractExpiry"] = MCXFuturesMeta_DF$ContractExpiry[i]
    if(identical(names(temp1),names(temp0)))
        {
        temp0 = rbind(temp0,temp1)
        }
    else
        {
        names(temp1) = names(temp0)
        temp0 = rbind(temp0,temp1)
        }
   
    msg = str_c("Dataset for ",k," is created. Its Contact Expiry month is ", MCXFuturesMeta_DF$ContractExpiry[i])
    print(msg)
}


[1] "Dataset for PBZ2013 is created. Its Contact Expiry month is December 2013"
[1] "Dataset for PBF2014 is created. Its Contact Expiry month is January 2014"
[1] "Dataset for PBG2014 is created. Its Contact Expiry month is February 2014"
[1] "Dataset for PBH2014 is created. Its Contact Expiry month is March 2014"
[1] "Dataset for PBJ2014 is created. Its Contact Expiry month is April 2014"
[1] "Dataset for PBM2014 is created. Its Contact Expiry month is June 2014"
[1] "Dataset for PBN2014 is created. Its Contact Expiry month is July 2014"
[1] "Dataset for PBQ2014 is created. Its Contact Expiry month is August 2014"
[1] "Dataset for PBU2014 is created. Its Contact Expiry month is September 2014"
[1] "Dataset for PBV2014 is created. Its Contact Expiry month is October 2014"
[1] "Dataset for PBX2014 is created. Its Contact Expiry month is November 2014"
[1] "Dataset for PBZ2014 is created. Its Contact Expiry month is December 2014"
[1] "Dataset for PBF2015 is created. Its Contact Expiry m

In [255]:
MCX_Master_Data = temp0

In [256]:
nrow(MCX_Master_Data)

[1] 4555

In [261]:
MCX_Master_Data = MCX_Master_Data[order(MCX_Master_Data$ContractDate,MCX_Master_Data$Date),]

#### Missing Value Treatment

In [269]:
MCX_Master_Data["WeekDay_Name"] = weekdays(MCX_Master_Data$Date)

##### Removing Saturdays

In [293]:
MCX_Master_Data = MCX_Master_Data[!(MCX_Master_Data$WeekDay_Name=="Saturday"),]

Before moving on to treatment for Missing values in "Open","Close","High" and "Low". Let us perform outlier treatment on volume

In [307]:
p = plot_ly(x=~MCX_Master_Data$Volume, type="box")

In [311]:
embed_notebook(p)

ERROR: Error in setwd(dir): cannot change working directory


In [313]:
dir

function (path = ".", pattern = NULL, all.files = FALSE, full.names = FALSE, 
    recursive = FALSE, ignore.case = FALSE, include.dirs = FALSE, 
    no.. = FALSE) 
.Internal(list.files(path, pattern, all.files, full.names, recursive, 
    ignore.case, include.dirs, no..))
<bytecode: 0x00000000093b5bc0>
<environment: namespace:base>

In [315]:
max(MCXFuturesMeta_DF$ContractDate)

[1] "2018-02-01"

In [329]:
as.integer(str_sub(as.character.Date(Sys.Date()),9))-1

[1] 0

In [379]:
MCX_Search_Results = Quandl.search("Lead Futures",per_page = 50,silent = TRUE)
MCXFuturesMeta_DF = data.frame(Name = MCX_Search_Results$name, Exchange = MCX_Search_Results$database_code,DataSetCode = MCX_Search_Results$dataset_code,ContractDate=as.Date(str_c('01 ',str_replace(word(MCX_Search_Results$name,3,4),",","")),format = '%d %B %Y'),ContractExpiry=str_replace(word(MCX_Search_Results$name,3,4),",",""))
MCXFuturesMeta_DF["ExpiryMonth"]=word(MCXFuturesMeta_DF$ContractExpiry,1)
MCXFuturesMeta_DF["ExpiryYear"]=word(MCXFuturesMeta_DF$ContractExpiry,2)
MCXFuturesMeta_DF["Code_Abbrv"] = str_sub(MCXFuturesMeta_DF$DataSetCode,1,3)
MCXFuturesMeta_DF = MCXFuturesMeta_DF[order(MCXFuturesMeta_DF$ContractDate),]


ERROR: Error in View(head(MCXFuturesMeta_DF)): 'View()' not yet supported in the Jupyter R kernel


In [380]:
MCX_Future_Code_Month_Mapping = unique(MCXFuturesMeta_DF[,c("ExpiryMonth","Code_Abbrv")])

In [383]:
MCX_Future_Code_Month_Mapping["Key"]=1

In [392]:
MCX_Future_Code_Month_Mapping

,ExpiryMonth,Code_Abbrv,Key
31,December,PBZ,12
41,January,PBF,1
13,February,PBG,2
14,March,PBH,3
42,April,PBJ,4
12,June,PBM,6
43,July,PBN,7
3,August,PBQ,8
15,September,PBU,9
1,October,PBV,10


In [388]:
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='December',"Key"]=12

In [390]:
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='November',"Key"]=11

In [391]:
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='February',"Key"]=2
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='March',"Key"]=3
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='April',"Key"]=4
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='May',"Key"]=5
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='June',"Key"]=6
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='July',"Key"]=7
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='August',"Key"]=8
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='September',"Key"]=9
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='October',"Key"]=10
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='November',"Key"]=11
MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$ExpiryMonth=='December',"Key"]=12

In [404]:
as.integer(str_sub(as.character.Date(max(MCXFuturesMeta_DF$ContractDate)),6,-4))+1

[1] 3

In [395]:
as.integer(str_sub(as.character.Date(max(Sys.Date())),6,-4))

[1] 4

In [405]:
for (k in c(3:4)){
    
    print(k)
}

[1] 3
[1] 4


In [406]:
MaximumAvailableMonth = as.integer(str_sub(as.character.Date(max(MCXFuturesMeta_DF$ContractDate)),6,-4))+1
CurrentMonth = as.integer(str_sub(as.character.Date(max(Sys.Date())),6,-4))
CurrentYear = str_sub(as.character.Date(max(Sys.Date())),1,-7)

In [419]:
str

[1] "2018"

In [450]:
newCode=""
i=1
for (m in c(MaximumAvailableMonth:CurrentMonth)){
    code_new = str_c(MCX_Future_Code_Month_Mapping[MCX_Future_Code_Month_Mapping$Key==m,"Code_Abbrv"],CurrentYear)
    newCode[i]=code_new
    print(code_new)
    i= i + 1
}

[1] "PBH2018"
[1] "PBJ2018"


[1] "PBH2018" "PBJ2018"

ERROR: Error in newCode.append("s"): could not find function "newCode.append"
